In [1]:
# ## consume to mongodb
# from kafka import KafkaConsumer
# import sys, json, pymongo
# # from elasticsearch import Elasticsearch
# # import time, datetime



# if __name__ == "__main__":

#     # 與 MongoDB連線
#     client = pymongo.MongoClient(host="mongodb", port=27017)
#     # 指定為 test 資料庫
#     db = client.test
#     # 指定 temp_humidity 集合, MongoDB的每個資料庫又包含許多集合(collection), 類似於關聯性資料庫中的表
#     collection = db.xyz

#     # 設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
#     consumer = KafkaConsumer(
#         # 指定Kafka集群伺服器
#         bootstrap_servers=["kafka:9092"],
#         # ConsumerGroup的名稱, 可以不指定
#         #group_id="cg_001",
#         # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
#         # key_deserializer=bytes.decode,
#         # 指定msgValue的反序列化器
#         #value_deserializer=bytes.decode,
#         value_deserializer=lambda m: json.loads(m.decode('ascii')),
#         # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
#         auto_offset_reset="earliest",
#     )
   
#     # 讓Consumer向Kafka集群訂閱指定的topic
#     consumer.subscribe(topics="location")
    
#     #es = Elasticsearch('http://elasticsearch:9200')
    
    
#     # 持續的拉取Kafka有進來的訊息
#     try:
#         print("Now listening for incoming messages ...")
#         # 持續監控是否有新的record進來
#         for record in consumer:
#             topic = record.topic
#             partition = record.partition
#             offset = record.offset
#             timestamp = record.timestamp
#             # 取出msgKey與msgValue
#             msgKey = record.key
#             msgValue = record.value
#             # 秀出metadata與msgKey & msgValue訊息
#             print("topic=%s, partition=%s, offset=%s : (key=%s, value=%s)" % (record.topic, record.partition, 
#                                                                               record.offset, record.key, record.value))
#             j = {"device_id": msgValue["device_id"], "timestamp": msgValue["timestamp"], 
#                  "longitude": msgValue["longitude"], "latitude": msgValue["latitude"], "altitude": msgValue["altitude"]}
#             print("json=",  j)
            
#             # 將資料存入 mongodb
#             # 存入單筆
#             result = collection.insert_one(j)
#             # 存入多筆
#             #result = collection.insert_many()
#             print(result)
#             #res = es.index(index="temp_humidity1", doc_type='tweet', id=datetime.now(), body=j)
#             #print(res['result'])

#     except:
#         # 錯誤處理
#         e_type, e_value, e_traceback = sys.exc_info()
#         print("type ==> %s" % (e_type))
#         print("value ==> %s" % (e_value))
#         print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
#         print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
#         print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
#     finally:
#         consumer.close()

In [ ]:
## consume to Elasticsearch
from kafka import KafkaConsumer
import sys, json
# import pymongo
from elasticsearch import Elasticsearch
import time, datetime




if __name__ == "__main__":

#     # 與 MongoDB連線
#     client = pymongo.MongoClient(host="mongodb", port=27017)
#     # 指定為 test 資料庫
#     db = client.test
#     # 指定 temp_humidity 集合, MongoDB的每個資料庫又包含許多集合(collection), 類似於關聯性資料庫中的表
#     collection = db.xyz

    # 設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
    consumer = KafkaConsumer(
        # 指定Kafka集群伺服器
        bootstrap_servers=["kafka:9092"],
        # ConsumerGroup的名稱, 可以不指定
        #group_id="cg_001",
        # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
        # key_deserializer=bytes.decode,
        # 指定msgValue的反序列化器
        #value_deserializer=bytes.decode,
        value_deserializer=lambda m: json.loads(m.decode('ascii')),
        # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
        auto_offset_reset="earliest",
    )
   
    # 讓Consumer向Kafka集群訂閱指定的topic
    consumer.subscribe(topics="location")
    
    es = Elasticsearch('http://elasticsearch:9200')
    
    
    # 持續的拉取Kafka有進來的訊息
    try:
        print("Now listening for incoming messages ...")
        # 持續監控是否有新的record進來
        for record in consumer:
            topic = record.topic
            partition = record.partition
            offset = record.offset
            timestamp = record.timestamp
            # 取出msgKey與msgValue
            msgKey = record.key
            msgValue = record.value
            # 秀出metadata與msgKey & msgValue訊息
            print("topic=%s, partition=%s, offset=%s : (key=%s, value=%s)" % (record.topic, record.partition, 
                                                                              record.offset, record.key, record.value))
            j = {"device_id": msgValue["device_id"], "timestamp": msgValue["timestamp"], 
                 "longitude": msgValue["longitude"], "latitude": msgValue["latitude"], "altitude": msgValue["altitude"]}
            print("json=",  j)
            
#             # 將資料存入 mongodb
#             # 存入單筆
#             result = collection.insert_one(j)
#             # 存入多筆
#             # result = collection.insert_many()
#             print(result)


            res = es.index(index="location", doc_type='tweet', id=datetime.datetime.now(), body=j)
            print(res['result'])

    except:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
    finally:
        consumer.close()

Now listening for incoming messages ...
topic=location, partition=0, offset=0 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:03:11', 'latitude': 24.968069166666666, 'longitude': 121.1915475, 'altitude': 142.7})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:03:11', 'longitude': 121.1915475, 'latitude': 24.968069166666666, 'altitude': 142.7}
created
topic=location, partition=0, offset=1 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:03:15', 'latitude': 24.968068, 'longitude': 121.19154716666667, 'altitude': 142.9})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:03:15', 'longitude': 121.19154716666667, 'latitude': 24.968068, 'altitude': 142.9}
created
topic=location, partition=0, offset=2 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:03:19', 'latitude': 24.968067666666666, 'longitude': 121.1915465, 'altitude': 143.3})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:03:19', 'longitude': 121.191546

json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:06:29', 'longitude': 121.191558, 'latitude': 24.9680655, 'altitude': 156.6}
created
topic=location, partition=0, offset=51 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:06:32', 'latitude': 24.9680675, 'longitude': 121.191558, 'altitude': 156.3})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:06:32', 'longitude': 121.191558, 'latitude': 24.9680675, 'altitude': 156.3}
created
topic=location, partition=0, offset=52 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:06:36', 'latitude': 24.968067833333333, 'longitude': 121.19156, 'altitude': 156.8})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:06:36', 'longitude': 121.19156, 'latitude': 24.968067833333333, 'altitude': 156.8}
created
topic=location, partition=0, offset=53 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:06:39', 'latitude': 24.968067166666668, 'longitude': 121.1915615, 'altitude': 157.2})
js

created
topic=location, partition=0, offset=90 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:08:58', 'latitude': 24.968040833333333, 'longitude': 121.191553, 'altitude': 144.5})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:08:58', 'longitude': 121.191553, 'latitude': 24.968040833333333, 'altitude': 144.5}
created
topic=location, partition=0, offset=91 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:09:01', 'latitude': 24.96804, 'longitude': 121.191552, 'altitude': 145.3})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:09:01', 'longitude': 121.191552, 'latitude': 24.96804, 'altitude': 145.3}
created
topic=location, partition=0, offset=92 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:09:04', 'latitude': 24.9680405, 'longitude': 121.1915515, 'altitude': 145.7})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:09:04', 'longitude': 121.1915515, 'latitude': 24.9680405, 'altitude': 145.7}
created
top

created
topic=location, partition=0, offset=137 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:11:49', 'latitude': 24.968035, 'longitude': 121.19154133333333, 'altitude': 136.7})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:11:49', 'longitude': 121.19154133333333, 'latitude': 24.968035, 'altitude': 136.7}
created
topic=location, partition=0, offset=138 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:11:53', 'latitude': 24.968034, 'longitude': 121.191541, 'altitude': 136.8})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:11:53', 'longitude': 121.191541, 'latitude': 24.968034, 'altitude': 136.8}
created
topic=location, partition=0, offset=139 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:11:56', 'latitude': 24.9680215, 'longitude': 121.19154966666666, 'altitude': 139.0})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:11:56', 'longitude': 121.19154966666666, 'latitude': 24.9680215, 'altitude': 1

created
topic=location, partition=0, offset=186 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:14:51', 'latitude': 24.968087666666666, 'longitude': 121.191525, 'altitude': 153.2})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:14:51', 'longitude': 121.191525, 'latitude': 24.968087666666666, 'altitude': 153.2}
created
topic=location, partition=0, offset=187 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:14:56', 'latitude': 24.968087833333332, 'longitude': 121.19152433333333, 'altitude': 153.2})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:14:56', 'longitude': 121.19152433333333, 'latitude': 24.968087833333332, 'altitude': 153.2}
created
topic=location, partition=0, offset=188 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 14:14:59', 'latitude': 24.968088833333333, 'longitude': 121.19152533333333, 'altitude': 153.0})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 14:14:59', 'longitude': 121.191525333

...

created
topic=location, partition=0, offset=2921 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 21:25:55', 'latitude': 24.968011833333332, 'longitude': 121.1914025, 'altitude': 149.9})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 21:25:55', 'longitude': 121.1914025, 'latitude': 24.968011833333332, 'altitude': 149.9}
created
topic=location, partition=0, offset=2922 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 21:26:03', 'latitude': 24.968014833333335, 'longitude': 121.19140316666666, 'altitude': 150.0})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 21:26:03', 'longitude': 121.19140316666666, 'latitude': 24.968014833333335, 'altitude': 150.0}
created
topic=location, partition=0, offset=2923 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 21:26:08', 'latitude': 24.968121166666666, 'longitude': 121.19141566666667, 'altitude': 157.2})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 21:26:08', 'longitude': 121.1914

created
topic=location, partition=0, offset=2980 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:51:18', 'latitude': 0.0, 'longitude': 0.0, 'altitude': None})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:51:18', 'longitude': 0.0, 'latitude': 0.0, 'altitude': None}
created
topic=location, partition=0, offset=2981 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:51:21', 'latitude': 0.0, 'longitude': 0.0, 'altitude': None})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:51:21', 'longitude': 0.0, 'latitude': 0.0, 'altitude': None}
created
topic=location, partition=0, offset=2982 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:51:23', 'latitude': 0.0, 'longitude': 0.0, 'altitude': None})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:51:23', 'longitude': 0.0, 'latitude': 0.0, 'altitude': None}
created
topic=location, partition=0, offset=2983 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-

topic=location, partition=0, offset=3006 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:52:41', 'latitude': 24.968022, 'longitude': 121.19151416666666, 'altitude': 138.9})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:52:41', 'longitude': 121.19151416666666, 'latitude': 24.968022, 'altitude': 138.9}
created
topic=location, partition=0, offset=3007 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:52:46', 'latitude': 24.968021666666665, 'longitude': 121.191525, 'altitude': 145.4})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:52:46', 'longitude': 121.191525, 'latitude': 24.968021666666665, 'altitude': 145.4}
created
topic=location, partition=0, offset=3008 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:52:51', 'latitude': 24.968027666666668, 'longitude': 121.19152666666666, 'altitude': 144.9})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:52:51', 'longitude': 121.19152666666666, 'latitude': 24.9

topic=location, partition=0, offset=3030 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:54:21', 'latitude': 24.967995333333334, 'longitude': 121.19151916666667, 'altitude': 144.0})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:54:21', 'longitude': 121.19151916666667, 'latitude': 24.967995333333334, 'altitude': 144.0}
created
topic=location, partition=0, offset=3031 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:54:25', 'latitude': 24.967996333333332, 'longitude': 121.1915185, 'altitude': 144.0})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:54:25', 'longitude': 121.1915185, 'latitude': 24.967996333333332, 'altitude': 144.0}
created
topic=location, partition=0, offset=3032 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:54:29', 'latitude': 24.967997333333333, 'longitude': 121.19151883333333, 'altitude': 144.3})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:54:29', 'longitude': 121.191518833333

topic=location, partition=0, offset=3054 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:55:59', 'latitude': 24.968000333333332, 'longitude': 121.19148483333333, 'altitude': 136.5})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:55:59', 'longitude': 121.19148483333333, 'latitude': 24.968000333333332, 'altitude': 136.5}
created
topic=location, partition=0, offset=3055 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:56:03', 'latitude': 24.968003, 'longitude': 121.19148383333334, 'altitude': 136.6})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:56:03', 'longitude': 121.19148383333334, 'latitude': 24.968003, 'altitude': 136.6}
created
topic=location, partition=0, offset=3056 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 11:56:07', 'latitude': 24.968005166666668, 'longitude': 121.19148283333334, 'altitude': 136.7})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 11:56:07', 'longitude': 121.19148283333334, 